In [1]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import time
import csv
import os
import json
import pandas as pd
import difflib

Please add the following missing content!
"HDP_Hawker and coffeshop listing_for online use_16.5.2018.xlsx" rename to "HDP_Hawker_and_coffeeshop.xlsx"

https://ch-api.healthhub.sg/api/public/content/2be093bf58c948bd8e510df83a80914a?v=ee49b3af

"List_of_Malls.txt" copy and pasted from https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore

In [3]:
# File paths
EXCEL_FILE = "../Data_Raw/HDP_Hawker_and_coffeeshop.xlsx"
CLEANED_HAWKER_CSV = "Hawker.csv"
HAWKER_COORDS_CSV = "../Data_Coordinates/Hawker.csv"

MALL_LIST_FILE = "../Data_Raw/List_of_Malls.txt"
MALL_COORDS_CSV = "../Data_Coordinates/MallCoordinates.csv"

FAILURE_LOG = "google_api_failures.json"
ADDRESS_CACHE_FILE = "address_cache.json"


In [ ]:
# Load Google Maps API Key from environment variables
GOOGLE_MAPS_API_KEY = "Google API Key"

GOOGLE_API_URL = "https://maps.googleapis.com/maps/api/geocode/json"
session = requests.Session()

# Core Functions

Loading and saving JSON file

In [5]:
def load_json_file(file_path):
    """Load JSON file if exists, otherwise return an empty dictionary."""
    return json.load(open(file_path, "r")) if os.path.exists(file_path) else {}


def save_json_file(file_path, data):
    """Save data to a JSON file."""
    with open(file_path, "w") as f:
        json.dump(data, f)

Clean the hawker dataset by removing duplications based on name of hawker centre/ coffee shop 

In [6]:
def clean_hawker_data():
    """Load and clean the hawker centre data from an Excel file."""
    print("📌 Cleaning hawker centre data...")

    df = pd.read_excel(EXCEL_FILE, usecols=["Name of hawker centre/coffee\nshop", "Address"], engine="openpyxl")
    df = df.drop_duplicates().dropna()
    df["Address"] = df["Address"].str.strip()

    df.to_csv(CLEANED_HAWKER_CSV, index=False, encoding="utf-8")
    print(f"✅ Cleaned data saved to {CLEANED_HAWKER_CSV} ({len(df)} rows)")

    return df

Fetch via google api

In [7]:
def get_coordinates_google(address, cache, failure_log):
    """Fetch latitude and longitude from Google Maps API with caching and failure handling."""
    if address in cache:
        return cache[address]

    if address in failure_log:
        return None, None

    params = {"address": f"{address}, Singapore", "region": "SG", "key": GOOGLE_MAPS_API_KEY}

    try:
        response = session.get(GOOGLE_API_URL, params=params, timeout=5)
        data = response.json()

        if response.status_code == 200 and data.get("results"):
            location = data["results"][0]["geometry"]["location"]
            cache[address] = (location["lat"], location["lng"])
            save_json_file(ADDRESS_CACHE_FILE, cache)
            return location["lat"], location["lng"]

        failure_log[address] = True
        save_json_file(FAILURE_LOG, failure_log)
        return None, None

    except requests.RequestException:
        failure_log[address] = True
        save_json_file(FAILURE_LOG, failure_log)
        return None, None

In [8]:
def fetch_coordinates(df, entity_name="Location"):
    """Fetch coordinates for each unique address using Google Maps API with progress tracking."""
    failure_log = load_json_file(FAILURE_LOG)
    cache = load_json_file(ADDRESS_CACHE_FILE)
    results = []

    total_addresses = len(df)
    start_time = time.time()

    for index, (_, row) in enumerate(df.iterrows(), start=1):
        name, address = row["Name"], row["Address"]
        lat, lon = get_coordinates_google(address, cache, failure_log)
        results.append([name, address, lat, lon])

        elapsed_time = time.time() - start_time
        estimated_time_remaining = (elapsed_time / index) * total_addresses - elapsed_time

        print(f"🟢 [{index}/{total_addresses}] {entity_name}: {name} | Lat: {lat}, Lon: {lon} "
              f"| Elapsed: {elapsed_time:.2f}s | ETA: {estimated_time_remaining:.2f}s")

        time.sleep(0.2)  # API rate limiting

    return results

Save CSV file

In [ ]:
def save_to_csv(csv_file, data, headers):
    """Save data to a CSV file."""
    with open(csv_file, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(data)
    print(f"✅ Data saved to {csv_file} ({len(data)} rows)")

Load mall from file .txt

In [10]:
def load_malls_from_file():
    """Read mall names and their regions from a text file."""
    regions = {}
    current_region = None
    with open(MALL_LIST_FILE, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line:
                if line in ["Central", "East", "North", "North East", "North West", "South", "West"]:
                    current_region = line
                    regions[current_region] = []
                elif current_region:
                    regions[current_region].append(line)
    return regions

In [11]:
def load_existing_data(csv_file):
    """Load existing mall data from a CSV file if available."""
    existing_data = {}
    if os.path.exists(csv_file):
        with open(csv_file, "r", encoding="utf-8") as file:
            reader = csv.reader(file)
            next(reader)  # Skip header
            for row in reader:
                mall_name, region, lat, lon = row
                existing_data[mall_name.lower()] = [mall_name, region, lat, lon]
    return existing_data


Fetch and update the mall csv

In [22]:
def process_malls(regions):
    """Process malls, reuse existing coordinates, and only fetch missing ones."""
    failure_log = load_json_file(FAILURE_LOG)
    cache = load_json_file(ADDRESS_CACHE_FILE)
    existing_mall_data = load_existing_data(MALL_COORDS_CSV)
    results = []

    malls_list = [(region, mall) for region, malls in regions.items() for mall in malls]
    total_malls = len(malls_list)
    start_time = time.time()

    for index, (region, mall) in enumerate(malls_list, start=1):
        mall_cleaned = mall.split("(")[0].strip()
        existing_match = difflib.get_close_matches(mall_cleaned.lower(), existing_mall_data.keys(), n=1, cutoff=0.8)

        if existing_match:
            mall_name, existing_region, lat, lon = existing_mall_data[existing_match[0]]
            if lat and lon:
                print(f"⏩ Skipping: {mall_cleaned} (Already in dataset)")
            else:
                lat, lon = get_coordinates_google(mall_cleaned, cache, failure_log)
        else:
            lat, lon = get_coordinates_google(mall_cleaned, cache, failure_log)

        results.append([mall_cleaned, region, lat, lon])

        elapsed_time = time.time() - start_time
        estimated_time_remaining = (elapsed_time / index) * total_malls - elapsed_time

        print(f"🟢 [{index}/{total_malls}] Mall: {mall_cleaned} | Lat: {lat}, Lon: {lon} "
              f"| Elapsed: {elapsed_time:.2f}s | ETA: {estimated_time_remaining:.2f}s")

        time.sleep(0.2)  # API rate limiting

    return results


# Fetch Hawker centre coordinates


In [21]:
print("📌 Cleaning and fetching hawker centre coordinates...")
df_hawker = clean_hawker_data()
df_hawker = df_hawker.rename(columns={"Name of hawker centre/coffee\nshop": "Name"})
hawker_results = fetch_coordinates(df_hawker, "Hawker Centre")
save_to_csv(HAWKER_COORDS_CSV, hawker_results, ["Hawker Centre / Coffeeshop", "Address", "Latitude", "Longitude"])

📌 Cleaning and fetching hawker centre coordinates...
📌 Cleaning hawker centre data...
✅ Cleaned data saved to Hawker.csv (961 rows)
🟢 [1/961] Hawker Centre: H23 Eating House | Lat: 1.3212494, Lon: 103.8866968 | Elapsed: 0.00s | ETA: 0.96s
🟢 [2/961] Hawker Centre: 117 Aljunied Market & Food
Centre | Lat: 1.3206162, Lon: 103.8869842 | Elapsed: 0.20s | ETA: 96.52s
🟢 [3/961] Hawker Centre: Sin Hin Eating House | Lat: 1.3200279, Lon: 103.8868022 | Elapsed: 0.40s | ETA: 128.32s
🟢 [4/961] Hawker Centre: Chang Cheng Food & Beverage | Lat: 1.3200279, Lon: 103.8868022 | Elapsed: 0.60s | ETA: 144.12s
🟢 [5/961] Hawker Centre: Lian Soon Reality Pte. Ltd | Lat: 1.3760127, Lon: 104.0028648 | Elapsed: 0.80s | ETA: 153.53s
🟢 [6/961] Hawker Centre: Yue Hua Food Place Pte. Ltd. | Lat: 1.3298365, Lon: 103.9309174 | Elapsed: 1.00s | ETA: 159.78s
🟢 [7/961] Hawker Centre: Lucky 5 | Lat: 1.3284617, Lon: 103.9306227 | Elapsed: 1.21s | ETA: 164.25s


KeyboardInterrupt: 

# Fetch Mall coordinates


In [23]:
print("📌 Processing malls and fetching coordinates...")
regions = load_malls_from_file()
mall_results = process_malls(regions)
save_to_csv(MALL_COORDS_CSV, mall_results, ["Mall Name", "Region", "Latitude", "Longitude"])


📌 Processing malls and fetching coordinates...
⏩ Skipping: 100 AM (Already in dataset)
🟢 [1/174] Mall: 100 AM | Lat: 1.2749931, Lon: 103.8435825 | Elapsed: 0.00s | ETA: 0.00s
⏩ Skipping: 313@Somerset (Already in dataset)
🟢 [2/174] Mall: 313@Somerset | Lat: 1.3010451, Lon: 103.8385792 | Elapsed: 0.20s | ETA: 17.31s
⏩ Skipping: Aperia (Already in dataset)
🟢 [3/174] Mall: Aperia | Lat: 1.310458, Lon: 103.8641669 | Elapsed: 0.40s | ETA: 22.92s
⏩ Skipping: Balestier Hill Shopping Centre (Already in dataset)
🟢 [4/174] Mall: Balestier Hill Shopping Centre | Lat: 1.3258546, Lon: 103.8428618 | Elapsed: 0.60s | ETA: 25.64s
⏩ Skipping: Bugis Cube (Already in dataset)
🟢 [5/174] Mall: Bugis Cube | Lat: 1.2981918, Lon: 103.8556572 | Elapsed: 0.80s | ETA: 27.20s
⏩ Skipping: Bugis Junction (Already in dataset)
🟢 [6/174] Mall: Bugis Junction | Lat: 1.2993754, Lon: 103.8555261 | Elapsed: 1.01s | ETA: 28.15s
⏩ Skipping: Bugis+ (Already in dataset)
🟢 [7/174] Mall: Bugis+ | Lat: 1.2995955, Lon: 103.8542191

KeyboardInterrupt: 